In [ ]:
import numpy as np
import os
import glob
import pandas as pd
import SimpleITK as sitk
import pandas as pd

In [ ]:
def get_3D_Dose_dif(pred, gt, possible_dose_mask=None):
    if possible_dose_mask is not None:
        pred = pred[possible_dose_mask > 0]
        gt = gt[possible_dose_mask > 0]

    dif = np.mean(np.abs(pred - gt))
    return dif

In [ ]:
root_path = "/Users/amithkamath/repo/deepdosesens"
data_path = os.path.join(root_path, "data", "processed-ONL")
pred_path = os.path.join(root_path, "data", "output-ONL", "output-ONL-6", "Prediction")
results_path = os.path.join(root_path, "results")

In [ ]:
# select all cases in the folder
cases = sorted(glob.glob(os.path.join(pred_path,"*")))

# define a list to store the dataframes for the dose metrics
dose_diff_data = {}
dose_mean_data = {}
pred_mean_data = {}
case_names = []

for case in cases:
    # select the case number
    case_nr = case.split("/")[-1]
    case_names.append(case_nr)
    
    # select and read the ground truth dose
    dose_gt_path = glob.glob(os.path.join(data_path, case_nr) + "/Dose.nii.gz")
    dose_gt = sitk.ReadImage(dose_gt_path[0])
    dose_gt = sitk.GetArrayFromImage(dose_gt)

    # select and read the predicted dose
    dose_pred_path = glob.glob(os.path.join(pred_path, case_nr) + "/Dose.nii.gz")
    dose_pred = sitk.ReadImage(dose_pred_path[0])
    dose_pred = sitk.GetArrayFromImage(dose_pred)

    # select all the structures in the folder
    structures = glob.glob(os.path.join(data_path, case_nr) + "/[!Dose]*")

    # define a list to store the dose metrics for the targets and oars
    dose_diff = {}
    dose_mean = {}
    pred_mean = {}
    names = []

    # For every structure calculate the DVHs and dose metrics
    for structure in structures:
        # select the name of the structure
        name = structure.split("/")[-1].split(".")[0]

        # If CT, ignore
        if name == "CT":
            continue
        
        # turn structure into a boolean mask
        struct = sitk.ReadImage(structure)
        mask = sitk.GetArrayFromImage(struct)
        mask = np.array(mask, dtype='bool')

        diff = get_3D_Dose_dif(dose_pred, dose_gt, possible_dose_mask=mask)
        mean_gt = np.mean(dose_gt[mask > 0])
        mean_pred = np.mean(dose_pred[mask > 0])

        # then compute the percentage of difference to the mean GT, and store in dict.
        dose_diff[name] = diff
        dose_mean[name] = mean_gt
        pred_mean[name] = mean_pred
        names.append(name)

    #subject_data = pd.DataFrame.from_dict(dose_percent, orient='index',
    #                   columns=names)
    dose_diff_data[case_nr] = dose_diff
    dose_mean_data[case_nr] = dose_mean
    pred_mean_data[case_nr] = pred_mean

subject_data = pd.DataFrame.from_dict(dose_diff_data)
subject_data.to_csv(os.path.join(root_path, "results", "ONL_mean_difference.csv"))

subject_data = pd.DataFrame.from_dict(dose_mean_data)
subject_data.to_csv(os.path.join(root_path, "results", "ONL_mean_gt_dose.csv"))

subject_data = pd.DataFrame.from_dict(pred_mean_data)
subject_data.to_csv(os.path.join(root_path, "results", "ONL_mean_pred_dose.csv"))